# P53 - DMS Analyse
#### by Frido Petersen, Dario Prifti, Maximilian Fidlin and Enno Schäfer
*With special thanks to our Co-Worker, inspiration and beloved friend: Chat-GPT*

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import seaborn as sns
import data_exploration as de
import data_cleanup as dc
import functions as fun
import Documentation as doc
import severity_score as ses
import scipy.stats as stats
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import AgglomerativeClustering

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'data_cleanup' has no attribute 'low_val'

In [ ]:
# All used data sets
gia_null_eto: pd.DataFrame = pd.read_csv('../DMS_data/P53_HUMAN_Giacomelli_NULL_Etoposide_2018.csv')
gia_null_nut: pd.DataFrame = pd.read_csv('../DMS_data/P53_HUMAN_Giacomelli_NULL_Nutlin_2018.csv')
gia_wt_nut: pd.DataFrame = pd.read_csv('../DMS_data/P53_HUMAN_Giacomelli_WT_Nutlin_2018.csv')
kot_hum: pd.DataFrame = pd.read_csv('../DMS_data/P53_HUMAN_Kotler_2018.csv')

aa = pd.read_csv('../DMS_data/aminoacids.csv')

## Grobe Struktur:
1) Data Cleanup: wir hatten diese Daten, das und das haben wir damit gemacht, so und so sehen unsere Daten jetzt aus
    (i) NA - remove
    (ii) Z - Transformierung
    (iii) Normalisierung
` `
` `
2) Data Exploration
    (i) Distanzmatrix
    (ii) Clustering
    (iii) weitere Clustering-Methoden
    (iv) Zusätzliche AS Daten
` `
` `
3) T-tests

## Comparibility of p53 Datasets
#### Finding similarities and differences in the 4 datasets on p53

In [ ]:
# giacomelli null etoposide
fun.hmap(doc.gia_null_eto_auf)

In [ ]:
# giacomelli wildtype nutlin
fun.hmap(doc.gia_wt_nut_auf)

In [ ]:
# giacomelli null nutlin
fun.hmap(doc.gia_null_nut_auf)

In [ ]:
# kotler
fun.hmap(doc.kot_hum_auf)

In [ ]:
fun.multiple_hmap(gia_null_eto, gia_null_nut, gia_wt_nut, kot_hum)

In [ ]:
# These heatmaps show different trends:
# What we need to consider is that the Kotler dataset only covers a range of amino acids from x to y. While the "Giacomelli wildtype nutlin" and "Giacomelli null nutlin" datasets exhibit some similarities in terms of trends and values, the same cannot be said when comparing them to the "Giacomelli null etoposide" dataset. This disparity is likely due to the use of different p53 activating agents, namely nutlin-3 and etoposide. One notable observation across all datasets is that amino acids in the range of approximately 100-300 generally display a negative effect caused by mutations. This could indicate a specific region that is evolutionary conserved an perfected. Additionally, the Kotler dataset exhibits a scarcity of values, which should be taken into consideration for future work.

In [ ]:
# In addition to visually comparing the datasets, I wanted to investigate whether the datasets share positions in the amino acid sequence where the sum of all DMS scores is the lowest. This would indicate that these specific locations are particularly conserved.

In [ ]:
# The 5 lowest values in the "Giacomelli null etoposide" dataset
dc.low_val(gia_null_eto, 5)

In [ ]:
# The 5 highest values in the "Giacomelli null etoposide" dataset
dc.high_val(gia_null_eto, 5)

In [ ]:
# The lowest Values (-> most affected by mutation) are found in:
doc.lowest_vals.head(20)

In [ ]:
#In general, we can also take a look at a flexible number of the most negative locations and take a look at them next to eachother
doc.lowest_vals_gesammelt.head(5)

IDEA: Maybe using T-Tests to show differences? In whatever context? -> idea inspired by project proposal from Malte and colleagues

## Data cleanup
#### Preparing the data to enable further anaylses

In [ ]:
# min max Normalisierung
norm_frame = dc.aufteilung_mut_pos(dc.norm(gia_null_eto))
print("Z-transfromation and Min Max normalisation of df")
fun.hmap(norm_frame)
print(f"Position of Low and High values of frame")
dc.min_max_val(norm_frame)

After we finished cleaning our data, we decided to transform the data into a new, more compact format.
In this new data frame the rows resemble the original AA sequence and the rows represent the exchange with a specific AA (e.g. A). The shown values are the DMS scores for the shown substitution. The NAs   shown for the exchanges where the old and new AA are the same, are changed to the value zero. With this transformed data set, further analyses are more easily to perform.

In [5]:
#THOUGHT: Maybe not putting it in the documentation but saving it in a variable to use it later on
dc.rmv_na(dc.df_transform(gia_null_eto))

position_mut,1,2,3,4,5,6,7,8,9,10,...,384,385,386,387,388,389,390,391,392,393
AS_old,M,E,E,P,Q,S,D,P,S,V,...,M,F,K,T,E,G,P,D,S,D
AS_new,,,,,,,,,,,,,,,,,,,,,
A,-0.788753,-0.376597,-0.178116,-1.175946,-0.345754,-0.733648,0.577294,0.057062,0.110162,0.505744,...,1.087271,0.592288,1.342046,0.781405,1.238938,1.169208,1.478115,1.036380,0.542344,1.132786
C,-1.969077,-0.068750,0.335770,0.191526,0.249694,0.218392,0.325113,0.046357,0.059489,0.879340,...,1.048168,1.011005,1.153228,1.162982,1.105490,1.394445,1.040421,1.202961,1.525051,0.458008
D,0.536895,0.180007,0.223665,0.431286,0.596059,0.364607,0.000000,0.404817,0.079720,0.951393,...,0.976694,1.248257,0.906721,1.104929,1.017745,0.959517,1.072748,0.000000,1.493992,0.000000
E,1.227243,0.000000,0.000000,0.752063,0.154363,0.451159,0.437353,0.477276,-0.026962,0.843035,...,1.396801,0.954702,0.514636,1.260646,0.000000,0.524010,1.000880,1.057979,1.592430,0.701935
F,0.536895,0.285277,-0.163861,0.143955,0.183432,0.022895,0.630453,-0.113914,-0.032559,0.631839,...,1.355672,0.000000,1.458921,1.086308,1.395992,0.678716,1.456029,1.748360,1.259045,1.576765
G,-1.053827,-1.136268,-1.231009,0.480629,0.672144,0.618498,0.042125,0.322946,0.149293,0.477052,...,1.601316,0.431552,0.694353,1.173473,0.887612,0.000000,0.547189,0.805634,1.739032,0.889272
H,-2.003616,0.018802,0.166849,0.346323,0.473020,0.351703,0.512018,0.386434,0.168393,0.618341,...,1.060380,1.696196,1.042860,1.040709,1.450329,1.054661,0.754332,0.979709,1.223101,1.077643
I,-1.370065,0.265437,-0.079066,0.655922,0.443878,0.504212,0.192564,0.072021,0.306439,0.494835,...,0.872244,0.984972,1.002368,0.885588,1.574588,1.302351,1.535477,1.636037,1.337679,1.860547


Max: Wir haben probiert Patientendaten zu bekommen, aber wir haben die nicht bekommen

## Data exploration
*Was sind die Ergebnisse, auf die ihr gekommen seid? Also implementierte Funktionen mit einfügen, um dann plotten zu können*

First, we wanted to get a feel for the mean scores for each substitution calculated from the whole length of the p53 protein.

In [ ]:
dc.rmv_na(de.mean_substitutions(gia_null_eto)) #--> hier noch nicht z-transformiert, normalisiert, etc...

Then we calculated the differences of the AA to each other to look for optimal and suboptimal interchanges.

In [11]:
# Distanzmatrix (DMS-Scores)
dist_p53 = de.dms_distance_matrix(gia_null_eto)

# IDEA: Compare Clustering with "dist_chem" ...

AS_old,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
AS_old,,,,,,,,,,,,,,,,,,,,
A,0.000000,5.118419,1.001127,0.950791,3.865506,1.093999,1.727396,5.869602,1.354001,1.858657,1.982457,1.449749,0.619728,1.422936,2.382516,0.879574,1.428805,3.844111,2.678816,5.224924
C,5.118419,0.000000,5.157128,5.225621,2.266408,4.671423,4.215585,2.799431,5.807398,4.018940,3.878780,4.844397,5.130724,6.064423,3.461968,5.276711,4.375534,2.457430,3.015105,2.006232
D,1.001127,5.157128,0.000000,0.981280,3.758420,1.082337,1.656670,6.041829,1.354857,2.016012,1.948455,1.268876,0.886359,1.481424,2.328884,0.981295,1.516787,3.841171,2.628414,5.168214
E,0.950791,5.225621,0.981280,0.000000,3.851885,0.958826,1.639293,5.929282,1.079586,1.820398,1.895125,1.302363,0.923046,1.366785,2.307710,1.034484,1.386531,3.846769,2.680332,5.261142
F,3.865506,2.266408,3.758420,3.851885,0.000000,3.462147,2.805199,3.117083,4.394297,2.539148,2.594098,3.492566,3.820806,4.605781,2.295875,3.973795,3.162375,1.985099,2.091920,2.351548
G,1.093999,4.671423,1.082337,0.958826,3.462147,0.000000,1.322393,5.478072,1.584928,1.630219,1.579012,1.216704,0.958718,1.847809,1.846426,1.157093,1.073299,3.380896,2.270993,4.789685
H,1.727396,4.215585,1.656670,1.639293,2.805199,1.322393,0.000000,5.192499,2.217342,1.468733,1.719725,1.441529,1.681892,2.346398,1.849399,1.679055,1.373227,3.283270,1.932457,4.188192
I,5.869602,2.799431,6.041829,5.929282,3.117083,5.478072,5.192499,0.000000,6.404813,4.593600,4.560649,5.714493,5.889325,6.628173,4.191128,6.130616,5.075546,2.830290,4.264680,3.487852
K,1.354001,5.807398,1.354857,1.079586,4.394297,1.584928,2.217342,6.404813,0.000000,2.334108,2.444159,1.628534,1.270695,1.173660,2.859190,1.243476,1.835478,4.438679,3.420946,5.984730


With this distance matrix for the whole length of our protein, we wanted to compare these findings with the distances based purely on the chemical properties.

In [ ]:
dist_chem = de.aa_distance_matrix(aa)

# IDEA: Compare Clustering with "dist_p53" ...

-> Gibt uns die Abstände der AS zueinander basierend auf den DMS-Scores bei Austausch von AS. Bin mir leider nicht sicher ob die Abstände der neuen oder alten AS zueinander berechnet werden. Und check auch nicht ganz wieso das nur mit der transponierten Matrix funktioniert

Anmerkung Enno, 29.6.: Frido, dann musst du das glaube ich nochmal schauen wie das genau geht :) Oder jemanden fragen...

## Domain comparison
#### Comparing Clusterings of substitutions in the context of specific protein domains

## Calculating severity scores
#### Matching DMS_scores with the mutation probability (only for single mutations)


In [ ]:
#slice RNA sequence in codons
p53_codons = [ses.rna_sequence[i:i+3] for i in range(0, len(ses.rna_sequence), 3)]

#generate all AA reachable by single mutation for the whole length of p53
p53_var_frame_raw: pd.DataFrame = ses.translate_codons_df(ses.generate_codon_variations(p53_codons))

#cleaning single mutations of STOP codons
p53_var_frame = ses.clean_variation_matrix(p53_var_frame_raw)

#prepare p53 dataset: might need transposing the df
df = dc.df_transform_inverse(norm_frame)
#select all single mutations from DMS and calculate severity score
sel_mut: pd.DataFrame = ses.select_smut(df, p53_var_frame)
severity_score_p53: pd.DataFrame = ses.prob_smut(sel_mut,p53_var_frame)


In [ ]:
# next -> which probable variations did show an increased fitness score? Is there an AMS with increased DMS score, that
# is not probable by single base Mutation?

#doing analysis on other dataframes
#comparing aa sequence from ensembl with used one of the dataset